In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

nba_data_2021_2022_advanced = pd.read_excel('2021_2022_Per_Game_Data.xlsx')
nba_data_2021_2022_per_game = pd.read_excel('2020_2021_NBA_Per_Game_Stats.xlsx')
nba_data_2020_2021_advanced = pd.read_excel('2020_2021_Player_Stats.xlsx')
nba_data_2020_2021_per_game = pd.read_excel('2021_22_NBA_Player_Data.xlsx')

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# Remove the 'MP' columns
nba_data_2021_2022_advanced.drop(columns=['MP'], inplace=True, errors='ignore')
nba_data_2021_2022_per_game.drop(columns=['MP'], inplace=True, errors='ignore')
nba_data_2020_2021_advanced.drop(columns=['MP'], inplace=True, errors='ignore')
nba_data_2020_2021_per_game.drop(columns=['MP'], inplace=True, errors='ignore')

# Combine datasets from the same seasons
combined_2021_2022 = pd.merge(nba_data_2021_2022_advanced, nba_data_2021_2022_per_game, on='Player')
combined_2020_2021 = pd.merge(nba_data_2020_2021_advanced, nba_data_2020_2021_per_game, on='Player')

# Standardize the numeric columns
scaler = StandardScaler()
numeric_columns_2020_2021 = combined_2020_2021.select_dtypes(include=['float64', 'int64']).columns
combined_2020_2021_standardized = combined_2020_2021.copy()
combined_2020_2021_standardized[numeric_columns_2020_2021] = scaler.fit_transform(combined_2020_2021[numeric_columns_2020_2021])

numeric_columns_2021_2022 = combined_2021_2022.select_dtypes(include=['float64', 'int64']).columns
combined_2021_2022_standardized = combined_2021_2022.copy()
combined_2021_2022_standardized[numeric_columns_2021_2022] = scaler.transform(combined_2021_2022[numeric_columns_2021_2022])

# Add 'Allstar' column to 2020-2021 dataset
all_stars_2020_2021 = [
    'List of All-Star players from the 2020-2021 season'
]
combined_2020_2021_standardized['Allstar'] = combined_2020_2021_standardized['Player'].apply(lambda x: 'yes' if x in all_stars_2020_2021 else 'no')

In [ ]:
# Prepare data for model training and testing
X_train = combined_2020_2021_standardized[numeric_columns_2020_2021]
y_train = combined_2020_2021_standardized['Allstar'].apply(lambda x: 1 if x == 'yes' else 0)
X_test = combined_2021_2022_standardized[numeric_columns_2021_2022]

# Train the RandomForest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on 2021-2022 dataset
y_pred = model.predict(X_test)

# Count predicted All-Stars
predicted_allstars_count = sum(y_pred)